In [ ]:
# pip install tensorflow

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [3]:
outflows = pd.read_csv('train_df.csv')
outflows.head()

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category,year,cleaned_memo,tfidf_21,tfidf_22,...,amount_odd,decile_1,decile_2,decile_3,decile_4,decile_5,decile_6,decile_7,decile_8,decile_9
0,1603,acc_4444,VALLEY CENTER M. UTIL BILL XXXXXX XXXXXX 000 M...,277.21,2018-01-04,GENERAL_MERCHANDISE,2018,valley center m. util bill 000 magrath elle,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1603,acc_4440,USPS PO XXXXXXXX XXXX ARN,291.20,2018-02-01,GENERAL_MERCHANDISE,2018,usps po arn,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1603,acc_4440,USPS PO XXXXXXXX XXXX ARN,6.70,2018-02-01,GENERAL_MERCHANDISE,2018,usps po arn,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1603,acc_4444,VALLEY CENTER M. UTIL BILL XXXXXX XXXXXX 000 M...,261.84,2018-02-06,GENERAL_MERCHANDISE,2018,valley center m. util bill 000 magrath elle,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1603,acc_4444,SEARS PAYMENT PAYMENT XXXXXX XXXXXXXXXXXXXXX L...,79.00,2018-02-15,GENERAL_MERCHANDISE,2018,sears payment payment lieschen magrath,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
outflows.columns

Index(['prism_consumer_id', 'prism_account_id', 'memo', 'amount',
       'posted_date', 'category', 'year', 'cleaned_memo', 'tfidf_21',
       'tfidf_22',
       ...
       'amount_odd', 'decile_1', 'decile_2', 'decile_3', 'decile_4',
       'decile_5', 'decile_6', 'decile_7', 'decile_8', 'decile_9'],
      dtype='object', length=115)

In [5]:
X = outflows.iloc[:, 8:]
y = outflows['category']

In [6]:
# Encode y
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

In [8]:
# Reduce the number of features using PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=50)  
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [9]:
# Clear memory before building the model
import gc
gc.collect()

20

In [9]:
from tensorflow.keras.regularizers import l2

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [11]:
# Define feedforward neural network 
model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)), 
    Dense(y_train.shape[1], activation='softmax') 
])

In [14]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model
history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=10,  
    batch_size=32 
)

Epoch 1/10


2024-11-15 08:48:31.459763: I external/local_xla/xla/service/service.cc:168] XLA service 0x7efe9499aa10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-15 08:48:31.459792: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-11-15 08:48:31.459800: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-11-15 08:48:31.459806: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (2): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-11-15 08:48:31.459812: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (3): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-11-15 08:48:31.474226: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to ena

32662/32662 [==============================] - 91s 3ms/step - loss: 1.0321 - accuracy: 0.5680 - val_loss: 1.0133 - val_accuracy: 0.5766
Epoch 2/10
32662/32662 [==============================] - 91s 3ms/step - loss: 0.9840 - accuracy: 0.5874 - val_loss: 0.9793 - val_accuracy: 0.5877
Epoch 3/10
32662/32662 [==============================] - 90s 3ms/step - loss: 0.9627 - accuracy: 0.5978 - val_loss: 0.9689 - val_accuracy: 0.5935
Epoch 4/10
32662/32662 [==============================] - 89s 3ms/step - loss: 0.9551 - accuracy: 0.6007 - val_loss: 0.9632 - val_accuracy: 0.5969
Epoch 5/10
32662/32662 [==============================] - 69s 2ms/step - loss: 0.9500 - accuracy: 0.6035 - val_loss: 0.9592 - val_accuracy: 0.5965
Epoch 6/10
32662/32662 [==============================] - 90s 3ms/step - loss: 0.9441 - accuracy: 0.6086 - val_loss: 0.9543 - val_accuracy: 0.6034
Epoch 7/10
32662/32662 [==============================] - 88s 3ms/step - loss: 0.9379 - accuracy: 0.6135 - val_loss: 0.9469 - val

In [17]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2f}")

8166/8166 [==============================] - 9s 1ms/step - loss: 0.9402 - accuracy: 0.6102
Test accuracy: 0.61


In [28]:
model1 = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4), #randomly disables neurons & reduces co-dependency to generalize better to unseen data
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(y_categorical.shape[1], activation='softmax')
])

In [29]:
# Compile model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
# Train the model
history = model1.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

Epoch 1/5
32662/32662 [==============================] - 146s 4ms/step - loss: 0.9883 - accuracy: 0.5888 - val_loss: 0.9212 - val_accuracy: 0.6143
Epoch 2/5
32662/32662 [==============================] - 144s 4ms/step - loss: 0.9280 - accuracy: 0.6187 - val_loss: 0.8995 - val_accuracy: 0.6320
Epoch 3/5
32662/32662 [==============================] - 141s 4ms/step - loss: 0.9127 - accuracy: 0.6285 - val_loss: 0.8878 - val_accuracy: 0.6385
Epoch 4/5
32662/32662 [==============================] - 153s 5ms/step - loss: 0.9044 - accuracy: 0.6324 - val_loss: 0.8862 - val_accuracy: 0.6381
Epoch 5/5
32662/32662 [==============================] - 143s 4ms/step - loss: 0.8983 - accuracy: 0.6359 - val_loss: 0.8774 - val_accuracy: 0.6423


In [19]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)),
    Dropout(0.4),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.4),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(y_categorical.shape[1], activation='softmax')
])

In [21]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

Epoch 1/5
32662/32662 [==============================] - 150s 5ms/step - loss: 1.0691 - accuracy: 0.5765 - val_loss: 1.0347 - val_accuracy: 0.5903
Epoch 2/5
32662/32662 [==============================] - 154s 5ms/step - loss: 1.0371 - accuracy: 0.5886 - val_loss: 1.0180 - val_accuracy: 0.5946
Epoch 3/5
32662/32662 [==============================] - 150s 5ms/step - loss: 1.0281 - accuracy: 0.5904 - val_loss: 1.0066 - val_accuracy: 0.5967
Epoch 4/5
32662/32662 [==============================] - 153s 5ms/step - loss: 1.0251 - accuracy: 0.5907 - val_loss: 1.0128 - val_accuracy: 0.5934
Epoch 5/5
32662/32662 [==============================] - 147s 4ms/step - loss: 1.0240 - accuracy: 0.5910 - val_loss: 1.0173 - val_accuracy: 0.5938


In [15]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import BatchNormalization

In [16]:
from tensorflow.keras.optimizers import Adam

In [57]:
# # Learning rate scheduler and early stopping callback

# def lr_schedule(epoch, lr):
#     if epoch < 10:
#         return lr
#     elif epoch < 20:
#         return lr * 0.1
#     else:
#         return lr * 0.01
        
# lr_scheduler = LearningRateScheduler(lr_schedule)

In [11]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

Your GPUs may run slowly with dtype policy mixed_float16 because they do not have compute capability of at least 7.0. Your GPUs:
  NVIDIA GeForce GTX 1080 Ti, compute capability 6.1 (x4)
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


2024-11-15 09:53:19.507400: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-15 09:53:19.507994: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-15 09:53:19.508558: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [74]:
from tensorflow.keras.layers import LeakyReLU

model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dense(128, activation='tanh'),
    Dense(64),
    BatchNormalization(),
    Dense(64, activation='tanh'),
    Dense(y_train.shape[1], activation='softmax')
])
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# Train model with larger batch size and mixed precision
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=128,
    callbacks=[early_stopping]
)

Epoch 1/20
8166/8166 [==============================] - 57s 7ms/step - loss: 0.9382 - accuracy: 0.6122 - val_loss: 0.8965 - val_accuracy: 0.6321
Epoch 2/20
8166/8166 [==============================] - 57s 7ms/step - loss: 0.8600 - accuracy: 0.6522 - val_loss: 0.8632 - val_accuracy: 0.6474
Epoch 3/20
8166/8166 [==============================] - 57s 7ms/step - loss: 0.8341 - accuracy: 0.6651 - val_loss: 0.8518 - val_accuracy: 0.6549
Epoch 4/20
8166/8166 [==============================] - 57s 7ms/step - loss: 0.8213 - accuracy: 0.6705 - val_loss: 0.8404 - val_accuracy: 0.6579
Epoch 5/20
8166/8166 [==============================] - 53s 6ms/step - loss: 0.8137 - accuracy: 0.6737 - val_loss: 0.8357 - val_accuracy: 0.6596
Epoch 6/20
8166/8166 [==============================] - 56s 7ms/step - loss: 0.8076 - accuracy: 0.6766 - val_loss: 0.8285 - val_accuracy: 0.6634
Epoch 7/20
8166/8166 [==============================] - 60s 7ms/step - loss: 0.8030 - accuracy: 0.6781 - val_loss: 0.8289 - val_ac

In [25]:
#using residual connections
from tensorflow.keras.layers import Add, Input
from tensorflow.keras.models import Model

input_layer = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(input_layer)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
residual = Dense(64)(input_layer)  # Skip connection, allows the output of a previous layer to bypass intermediate layers
x = Add()([x, residual])
x = Dense(32, activation='relu')(x)
output = Dense(y_train.shape[1], activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=128,
    callbacks=[early_stopping]
)

Epoch 1/5
8166/8166 [==============================] - 277s 34ms/step - loss: 0.7413 - accuracy: 0.7106 - val_loss: 0.7129 - val_accuracy: 0.7185
Epoch 2/5
8166/8166 [==============================] - 265s 32ms/step - loss: 0.7065 - accuracy: 0.7208 - val_loss: 0.7022 - val_accuracy: 0.7219
Epoch 3/5
8166/8166 [==============================] - 263s 32ms/step - loss: 0.6987 - accuracy: 0.7235 - val_loss: 0.6980 - val_accuracy: 0.7235
Epoch 4/5
8166/8166 [==============================] - 261s 32ms/step - loss: 0.6944 - accuracy: 0.7250 - val_loss: 0.6973 - val_accuracy: 0.7241
Epoch 5/5
8166/8166 [==============================] - 261s 32ms/step - loss: 0.6917 - accuracy: 0.7256 - val_loss: 0.6938 - val_accuracy: 0.7251


In [25]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [22]:
#using residual connections
from tensorflow.keras.layers import Add, Input
from tensorflow.keras.models import Model

input_layer = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(input_layer)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
residual = Dense(64)(input_layer)  # Skip connection, allows the output of a previous layer to bypass intermediate layers
x = Add()([x, residual])
x = Dense(32, activation='relu')(x)
output = Dense(y_train.shape[1], activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=128,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/5
8166/8166 [==============================] - 53s 6ms/step - loss: 0.9537 - accuracy: 0.6043 - val_loss: 0.9076 - val_accuracy: 0.6282 - lr: 0.0010
Epoch 2/5
8166/8166 [==============================] - 51s 6ms/step - loss: 0.8636 - accuracy: 0.6509 - val_loss: 0.8655 - val_accuracy: 0.6489 - lr: 0.0010
Epoch 3/5
8166/8166 [==============================] - 51s 6ms/step - loss: 0.8334 - accuracy: 0.6652 - val_loss: 0.8473 - val_accuracy: 0.6570 - lr: 0.0010
Epoch 4/5
8166/8166 [==============================] - 51s 6ms/step - loss: 0.8174 - accuracy: 0.6727 - val_loss: 0.8356 - val_accuracy: 0.6637 - lr: 0.0010
Epoch 5/5
8166/8166 [==============================] - 51s 6ms/step - loss: 0.8050 - accuracy: 0.6781 - val_loss: 0.8209 - val_accuracy: 0.6714 - lr: 0.0010


In [24]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2f}")

8166/8166 [==============================] - 20s 2ms/step - loss: 0.8209 - accuracy: 0.6714
Test accuracy: 0.67
